# Lab2 

## import all the needed functions

In [9]:
import cv2 as cv
import random
import numpy as np
import sklearn as sk
import math
from matplotlib import pyplot as plt

## Import images 

In [10]:
# change to baboon image:

# image_name = "baboon.jpg"
# image = cv.imread(image_name)
image_name = "SAFlag.jpg"
image = cv.imread(image_name)

## make the global callback function

In [11]:
def call_back(*args):
    global k_value, lock, image_name,windowName
    k_value = cv.getTrackbarPos("K value", "Pic1")

## function to calculate the difference

In [12]:
def diff(x, y):      
    return math.sqrt(math.pow(int(x)-int(y), 2))


def color_diff(i, j):
        temp = diff(i[0], j[0])+diff(i[1], j[1])+diff(i[2], j[2])  
        return temp

## Random Center Initialization based on the number of clusters 

In [13]:
def random_center(k_value):
    seeds = []
    for j in range(k_value):
        seeds.append([[random.randint(0,image.shape[0]-1),random.randint(0,image.shape[1]-1)]])
        seeds[j].append(image[seeds[j][0][0]][seeds[j][0][1]].copy())
    return seeds

## Voronoi Region Segmentation based on the set of cluster centers

In [14]:
def voronoi_region_segmentation(seeds,image,threshold, k_value, upper_bound,lock):
    
    lock = True
    image_size = [image.shape[0],image.shape[1]]
    cluster = np.zeros((image_size[0],image_size[1]))
    go = True

    number_of_round = 0

    while go and number_of_round < upper_bound:
        print("Start round ", number_of_round + 1, "\n")
        number_of_round += 1

        go = False
        for x in range(image_size[0]):
            for y in range(image_size[1]):
                lower_bound = 100000
                min_value = 0
                for i in range(k_value):
                    temp = color_diff(seeds[i][1], image[x][y])
                    if temp < lower_bound  and diff(temp, lower_bound) > threshold :
                        lower_bound = temp
                        min_value = i
                cluster[x,y] = min_value
        
        mean = []
        for j in range(k_value):
            position = np.nonzero(cluster[:,:] == j)
            cluster_value = image[position]
            mean.append(np.mean(cluster_value, axis=0))
        for i in range(k_value):
            if  color_diff(seeds[i][1], mean[i]):
                go = True
            seeds[i][1] = mean[i]
    BGR(seeds,cluster,image,image_size)

## KMeans which calls Voronoi Region Segmentation

In [15]:
def kMeans(image,lock,image_name):
    if lock == True:
        Print("On going calculation, please wait")
        return 
    k_value = cv.getTrackbarPos("k Value", "Pic1")
    seeds = random_center(k_value)
    threshold = 5
    upper_bound = 10
    print("Start with k value of ",k_value, "on Image ",image_name,"\n")
    voronoi_region_segmentation(seeds, image, threshold, k_value, upper_bound,lock)

def BGR(seeds, cluster, image, image_size):
    for x in range(image_size[0]):
        for y in range(image_size[1]):
            idx = int(cluster[x,y])
            image[x,y] = seeds[idx][1] 
    cv.imshow(windowName, image)
    print(10*'!','finished',10*'!')
    lock = False

## Main Function

In [16]:
k_value = 0
windowName = 'Pic1'
cv.namedWindow(windowName, cv.WINDOW_NORMAL)
cv.resizeWindow(windowName, image.shape[1], image.shape[0])

cv.createTrackbar("k Value", windowName, 0, 30, call_back)
lock = False
print("After pick a K value, Press Enter to Start\n\n\n",20*'=')
cv.imshow(windowName, image)
while True:
    #cv.imshow("baboon.jpg", image)

    key=cv.waitKey(1)
    if key==27:
        break
    if key==13:    #press enter to start 
        kMeans(image,lock,image_name)
cv.destroyAllWindows()

After pick a K value, Press Enter to Start


Start with k value of  6 on Image  SAFlag.jpg 

Start round  1 

Start round  2 

Start round  3 

Start round  4 

Start round  5 

Start round  6 

Start round  7 

Start round  8 

Start round  9 

Start round  10 

!!!!!!!!!! finished !!!!!!!!!!
